# Brain Encoding with RL Features

## Predicting Brain Activity from Agent Representations

**Overview:**
This notebook uses the CNN activations from the RL agent (notebook 02) to predict brain activity during gameplay.

**What we'll cover:**
1. Understanding the encoding model framework
2. Loading and preparing BOLD data
3. Loading CNN activations from the agent
4. Aligning timepoints between BOLD and activations
5. Fitting ridge regression encoding models
6. Comparing layer performance
7. Visualizing brain maps

**Key question:** Which layer of the agent best predicts brain activity, and where?

In [ ]:
# @title Environment Setup (Colab & Local)
# @markdown This cell detects your environment and sets up the necessary dependencies and paths.
# @markdown On Colab, it will mount Google Drive to persist datasets and avoid re-downloading.

import sys
import os
import subprocess
from pathlib import Path

# Detect Colab
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("🚀 Detected Google Colab Environment")
    print("-----------------------------------------")

    # 1. Mount Google Drive
    try:
        from google.colab import drive
        print("📂 Mounting Google Drive...")
        drive.mount('/content/drive')
    except ImportError:
        print("⚠️ Google Drive not available. Using local runtime storage (will be lost on disconnect).")

    # Define project path
    # If Drive is available, use it. Otherwise use /content
    if os.path.exists('/content/drive/MyDrive'):
        DRIVE_ROOT = Path("/content/drive/MyDrive")
        PROJECT_NAME = "mario_tutorials"
        PROJECT_PATH = DRIVE_ROOT / PROJECT_NAME
    else:
        PROJECT_PATH = Path("/content/mario_tutorials")

    # 2. Setup Project Directory
    if not PROJECT_PATH.exists():
        print(f"🆕 Creating project directory at {PROJECT_PATH}...")
        PROJECT_PATH.mkdir(parents=True, exist_ok=True)
        
        print("⏬ Cloning repository...")
        try:
            subprocess.run(["git", "clone", "https://github.com/courtois-neuromod/mario.tutorials.git", str(PROJECT_PATH)], check=True)
        except subprocess.CalledProcessError:
            # Fallback if directory is not empty or git fails
            print("⚠️ Clone failed (maybe directory exists?). Attempting to pull...")
            os.chdir(PROJECT_PATH)
            subprocess.run(["git", "pull"], check=False)
    else:
        print(f"✅ Found existing project at {PROJECT_PATH}")

    # Change working directory
    os.chdir(PROJECT_PATH)
    print(f"📍 Working directory: {os.getcwd()}")

    # 3. Install System Dependencies
    # git-annex is required for DataLad
    # cmake and zlib1g-dev are required to build stable-retro (gym-retro fork)
    print("📦 Installing system dependencies (git-annex, cmake, zlib)...")
    subprocess.run(["apt-get", "update"], check=True)
    subprocess.run(["apt-get", "install", "-y", "git-annex", "cmake", "zlib1g-dev"], check=True)

    # 4. Install Python Dependencies
    print("🐍 Installing Python packages from requirements.txt...")
    if os.path.exists("requirements.txt"):
         # Using pip install -r is more robust
         # We add --verbose to see what's happening if it fails
         try:
             subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], check=True)
         except subprocess.CalledProcessError as e:
             print("❌ Pip install failed. Attempting to install core packages individually as fallback...")
             # Fallback: install critical packages one by one to isolate the issue
             packages = ["datalad", "nilearn", "gym", "stable-retro", "pandas", "scipy", "matplotlib", "seaborn", "h5py", "invoke"]
             for pkg in packages:
                 print(f"   Installing {pkg}...")
                 subprocess.run([sys.executable, "-m", "pip", "install", pkg], check=False)
    else:
         print("⚠️ requirements.txt not found! Installing default packages...")
         subprocess.run([sys.executable, "-m", "pip", "install", "datalad", "nilearn", "gym-retro", "ipympl", "pandas", "scipy", "matplotlib", "seaborn", "h5py", "invoke"], check=True)
    
    # 5. Add src to path
    if str(PROJECT_PATH / "src") not in sys.path:
        sys.path.append(str(PROJECT_PATH / "src"))
    
    print("✅ Environment setup complete!")

else:
    print("🖥️ Running Locally")
    # Ensure src is in path
    if str(Path.cwd() / "src") not in sys.path:
        sys.path.append(str(Path.cwd() / "src"))
    print("✅ Local environment assumed ready.")


In [ ]:
# Silent Setup
try:
    from setup_utils import setup_all
    # Ensure data is available (silently checks)
    setup_all(subject="sub-01", session="ses-010")
except ImportError:
    print("Setup utils not found. Please ensure src is in path.")
except Exception as e:
    print(f"Setup warning: {e}")


In [1]:
# Setup - imports and configuration

import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Add src to path
src_dir = Path('..') / 'src'
sys.path.insert(0, str(src_dir))

# Import utilities
from utils import (
    get_sourcedata_path,
    load_events,
    get_session_runs,
    load_bold
)

# Import RL utilities
from rl_utils import (
    create_simple_proxy_features,
    convolve_with_hrf,
    apply_pca
)

# Import RL visualizations
from rl_viz_utils import (
    plot_pca_variance_per_layer,
    plot_layer_activations_sample
)

# Import encoding utilities
from encoding_utils import (
    load_and_prepare_bold,
    fit_encoding_model_per_layer,
    compare_layer_performance
)

# Import encoding visualizations
from encoding_viz_utils import (
    plot_layer_comparison_bars,
    plot_r2_brainmap
)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Get sourcedata path
sourcedata_path = get_sourcedata_path()

print("✓ Setup complete!")

✓ Setup complete!


## 1. The Encoding Model Framework

**Goal:** Predict BOLD activity from RL agent features

**Model:** Ridge Regression (linear regression with L2 regularization)

```
BOLD(voxel, time) = Σ βᵢ · Feature_i(time) + ε
```

**Why ridge regression?**
- Handles high-dimensional features (50 PCA components)
- L2 penalty prevents overfitting: `||β||² ≤ α`
- Cross-validation selects optimal regularization strength α
- Fast to fit (~5 mins for whole brain)

**Alternative approaches:**
- Lasso (L1): Sparse feature selection
- Elastic net: L1 + L2
- Nonlinear: Kernel ridge, neural networks

**For interpretability and speed, we use ridge regression.**

In [2]:
# Load prerequisites

from nilearn.masking import compute_multi_epi_mask

# Define constants (assumed from the first tutorial)
SUBJECT = 'sub-01'
SESSION = 'ses-010'
TR = 1.49


# Get runs
runs = get_session_runs(SUBJECT, SESSION, sourcedata_path)
print(f"Found {len(runs)} runs: {runs}")

# Load events
all_events = []
for run in runs:
    events = load_events(SUBJECT, SESSION, run, sourcedata_path)
    all_events.append(events)
    print(f"  {run}: {len(events)} events")

# Create common mask (or load from main tutorial)
print("\nCreating common brain mask...")
bold_imgs= []
for run in runs:
    bold_img = load_bold(SUBJECT, SESSION, run, sourcedata_path)
    bold_imgs.append(bold_img)

common_mask = compute_multi_epi_mask(bold_imgs, n_jobs=1)
n_voxels = int((common_mask.get_fdata() > 0).sum())
print(f"✓ Common mask: {n_voxels:,} voxels")

print("\n✓ All prerequisites loaded!")


Found 4 runs: ['run-1', 'run-2', 'run-3', 'run-4']
  run-1: 953 events
  run-2: 986 events
  run-3: 892 events
  run-4: 1033 events

Creating common brain mask...
✓ Common mask: 213,443 voxels

✓ All prerequisites loaded!


## 2. Loading Prerequisites

We need:
- Subject/session info (sub-01, ses-010)
- Run IDs (4 runs)
- BOLD images (preprocessed fMRI data)
- Event files (for alignment)
- Common brain mask (from GLM analysis)

**Note:** If you haven't run notebook 01, this will create a fresh mask.

In [ ]:
# Load and align activations from replays

# First, check if we have a trained model
from pathlib import Path

MODEL_DIR = Path('../models')
MODEL_PATH = MODEL_DIR / 'mario_ppo_agent.pth'

if not MODEL_PATH.exists():
    print(f"✗ No trained model found at: {MODEL_PATH}")
    print("\nYou need a trained RL agent to extract activations.")
    print("Please train an agent first by running:")
    print("  python ../train_mario_agent.py --steps 5000000")
    print("\n⚠ Cannot proceed with encoding analysis without trained model")
    HAS_MODEL = False
else:
    print(f"✓ Found trained model: {MODEL_PATH}")
    HAS_MODEL = True
    
    # Load the model
    from rl_utils import load_pretrained_model, align_activations_to_bold
    
    print("\nLoading model...")
    model = load_pretrained_model(MODEL_PATH, device='cpu')
    print("✓ Model loaded")
    
    # Align activations to BOLD
    # This will:
    # 1. Load replay files for each game segment
    # 2. Extract RL activations at 60Hz  
    # 3. Downsample to TR (1.49s)
    # 4. Apply HRF convolution
    # 5. Create NaN mask for non-gameplay periods
    
    alignment_results = align_activations_to_bold(
        model=model,
        subject=SUBJECT,
        session=SESSION,
        runs=runs,
        sourcedata_path=sourcedata_path,
        tr=TR,
        device='cpu',
        apply_hrf=True  # Apply HRF convolution
    )
    
    # Extract results
    layer_activations = alignment_results['activations']
    valid_mask = alignment_results['mask']
    run_info = alignment_results['run_info']
    
    print(f"\n{'='*70}")
    print("Alignment summary:")
    for info in run_info:
        print(f"  {info['run']}: {info['n_valid_trs']}/{info['n_trs']} TRs "
              f"({info['n_segments']} game segments)")
    print(f"{'='*70}\n")


## 3. Loading and Aligning RL Activations

**NEW APPROACH:**

Instead of using pre-extracted activations, we now:

1. **Load replay files** from the human subject's actual gameplay
   - Uses `.bk2` replay files from `sourcedata/mario/`
   - Matches exact stimuli presented during fMRI scanning

2. **Extract activations frame-by-frame** (60Hz)
   - Pass replay frames through trained RL agent
   - Collect CNN activations from all layers

3. **Align to fMRI timing**
   - Use `mario.annotations` files to get game segment timing
   - Downsample from 60Hz to TR (1.49s)
   - Apply HRF convolution

4. **Handle multiple games per run**
   - Concatenate gameplay segments
   - Mask inter-game periods with NaN

**This ensures perfect alignment between RL activations and BOLD data!**

In [4]:
# Clean and prepare BOLD data

from encoding_utils import load_and_prepare_bold

print("Cleaning BOLD (detrending, standardizing)...\n")

bold_data = load_and_prepare_bold(
    bold_imgs,
    mask_img=common_mask,
    confounds_list=None,  # Already cleaned in GLM
    detrend=True,
    standardize=True,
    high_pass=1/128,
    t_r=TR
)

print(f"✓ BOLD prepared:")
print(f"  Shape: {bold_data.shape}")
print(f"  Timepoints: {bold_data.shape[0]}")
print(f"  Voxels: {bold_data.shape[1]:,}")

Cleaning BOLD (detrending, standardizing)...

✓ BOLD prepared:
  Shape: (1799, 213443)
  Timepoints: 1799
  Voxels: 213,443


## 4. Cleaning and Preparing BOLD Data

**Preprocessing steps:**
1. **Detrending:** Remove linear drift within each run
2. **High-pass filtering:** Remove slow fluctuations (<1/128 Hz)
3. **Standardization:** Z-score each voxel (mean=0, std=1)

**Why clean BOLD?**
- Scanner drift confounds encoding
- Slow fluctuations unrelated to task
- Standardization ensures comparable scales

**Output:** `(timepoints × voxels)` matrix ready for regression

In [ ]:
# Check alignment between BOLD and activations

if HAS_MODEL:
    n_bold = bold_data.shape[0]
    n_acts = list(layer_activations.values())[0].shape[0]
    
    print(f"BOLD timepoints: {n_bold}")
    print(f"Activations timepoints: {n_acts}")
    print(f"Valid (gameplay) timepoints: {valid_mask.sum()}")
    print(f"Invalid (non-gameplay) timepoints: {(~valid_mask).sum()}")
    
    # Ensure dimensions match
    if n_bold != n_acts:
        print(f"\n⚠ Dimension mismatch!")
        print(f"  Truncating to minimum length: {min(n_bold, n_acts)}")
        n_time = min(n_bold, n_acts)
        bold_data = bold_data[:n_time]
        valid_mask = valid_mask[:n_time]
        for layer in layer_activations.keys():
            layer_activations[layer] = layer_activations[layer][:n_time]
    else:
        print("\n✓ Dimensions match!")
else:
    print("⚠ No model available, skipping alignment check")


## 5. Alignment Status

**Automatic alignment completed!**

The `align_activations_to_bold()` function has:

1. ✅ **Loaded replay files** for each game segment
2. ✅ **Extracted RL activations** at 60Hz from replay frames
3. ✅ **Downsampled to TR** using temporal averaging within each TR window
4. ✅ **Applied HRF convolution** to account for hemodynamic lag
5. ✅ **Created validity mask** to mark gameplay vs non-gameplay periods

**Key differences from old approach:**
- OLD: Arbitrary agent gameplay, misaligned
- NEW: Exact subject gameplay from replays, perfectly aligned

**Dimensions should now match:**
- BOLD: Number of TRs across all runs
- Activations: Same number of TRs (with NaN for non-gameplay)

In [ ]:
# Create train/test split (only on valid gameplay TRs)

if HAS_MODEL:
    # Get valid indices
    valid_indices = np.where(valid_mask)[0]
    n_valid = len(valid_indices)
    
    # Split valid indices 80/20
    n_train_valid = int(n_valid * 0.8)
    
    train_valid_indices = valid_indices[:n_train_valid]
    test_valid_indices = valid_indices[n_train_valid:]
    
    print(f"Train/test split on valid (gameplay) TRs:")
    print(f"  Train: {len(train_valid_indices)} TRs")
    print(f"  Test: {len(test_valid_indices)} TRs")
    print(f"\nNote: Invalid (non-gameplay) TRs are excluded from both train and test")
else:
    print("⚠ No model available, skipping train/test split")


## 6. Applying PCA to Layer Activations

**Why PCA?**
- Raw activations have thousands of features per layer
- PCA reduces dimensionality while preserving variance
- Makes encoding models more tractable and prevents overfitting

**NaN handling:**
- PCA is fit only on valid (gameplay) TRs
- Invalid TRs remain NaN in the reduced activations
- This ensures proper alignment with BOLD data

**Target:** 50 PCA components per layer (or fewer if needed)

In [ ]:
# Apply PCA to layer activations

if HAS_MODEL:
    from rl_utils import apply_pca_with_nan_handling
    
    print("Applying PCA to reduce dimensionality...")
    print("(PCA is fit only on valid gameplay TRs)\n")
    
    pca_results = apply_pca_with_nan_handling(
        layer_activations,
        valid_mask,
        n_components=50,
        variance_threshold=0.9
    )
    
    # Extract reduced activations
    reduced_activations = pca_results['reduced_activations']
    pca_models = pca_results['pca_models']
    variance_explained = pca_results['variance_explained']
    
    print(f"\n{'='*70}")
    print("PCA summary:")
    for layer, acts in reduced_activations.items():
        print(f"  {layer}: {acts.shape[1]} components")
    print(f"{'='*70}\n")
else:
    print("⚠ No model available, skipping PCA")


## 7. Fitting Ridge Regression Encoding Models

**For each layer:**
1. Use PCA-reduced activations (50 components)
2. Cross-validate to find optimal α (regularization strength)
3. Fit ridge regression on training data (gameplay TRs only)
4. Predict BOLD on test data
5. Compute R² per voxel

**Hyperparameter search:** α ∈ [0.1, 1, 10, 100, 1000, 10000, 100000]

**NaN handling:**
- Only valid (gameplay) TRs are used for training and testing
- Invalid TRs are automatically excluded

**Output per layer:**
- Best α
- R² map: `(voxels,)` array
- Trained model

**Runtime:** ~5-10 minutes for all 5 layers × 213k voxels

**Interpretation:**
- R² > 0: Features explain variance in BOLD
- R² ≈ 0: No prediction (chance level)
- Negative R²: Worse than mean baseline

In [ ]:
# Fit ridge regression encoding models

if HAS_MODEL:
    from encoding_utils import fit_encoding_model_per_layer
    
    alphas = [0.1, 1, 10, 100, 1000, 10000, 100000]
    
    print("Fitting ridge regression (5 layers × voxels)...")
    print("This takes ~5-10 minutes\n")
    
    encoding_results = fit_encoding_model_per_layer(
        reduced_activations,
        bold_data,
        common_mask,
        train_valid_indices,
        test_valid_indices,
        alphas=alphas,
        valid_mask=valid_mask  # Pass valid mask for NaN handling
    )
    
    print("\n✓ Encoding complete!")
else:
    print("⚠ No model available, skipping encoding")

In [ ]:
# Compare layer performance

if HAS_MODEL:
    from encoding_utils import compare_layer_performance
    from encoding_viz_utils import plot_layer_comparison_bars
    
    comparison_df = compare_layer_performance(encoding_results)
    
    print("Layer Performance:\n")
    print("=" * 80)
    print(comparison_df.to_string(index=False))
    print("=" * 80)
    
    best_layer = comparison_df.iloc[0]['layer']
    best_r2 = comparison_df.iloc[0]['mean_r2']
    
    print(f"\n⭐ Best: {best_layer.upper()} (R² = {best_r2:.4f})")
    
    # Visualize
    layer_order = ['conv1', 'conv2', 'conv3', 'conv4', 'linear']
    fig = plot_layer_comparison_bars(encoding_results, layer_order)
    plt.show()
else:
    print("⚠ No model available, skipping layer comparison")

## 8. Comparing Layer Performance

**Question:** Which CNN layer best predicts brain activity?

**Metrics:**
- Mean R² (test set)
- Median R² (robust to outliers)
- % voxels with R² > 0.01 (significantly predicted)

**Expected pattern (if hypothesis holds):**
- Early layers (conv1/2) → Visual cortex
- Middle layers (conv3/4) → Motor/parietal
- Late layers (linear) → Frontal/executive

**See bar plot below for comparison.**

In [ ]:
# Visualize R² brain maps (best layer)

if HAS_MODEL:
    from encoding_viz_utils import plot_r2_brainmap
    
    best_layer = comparison_df.iloc[0]['layer']
    best_r2_map = encoding_results[best_layer]['r2_map']
    
    print(f"Best layer: {best_layer.upper()}\n")
    
    fig = plot_r2_brainmap(
        best_r2_map, 
        best_layer,
        threshold=0.01,
        vmax=0.2
    )
    plt.show()
    
    print("\n📍 Interpretation:")
    print("  Hot regions = Well predicted by this layer")
    print("  - Early layers → Visual cortex")
    print("  - Middle layers → Motor/parietal")
    print("  - Late layers → Frontal/executive")
else:
    print("⚠ No model available, skipping brain map visualization")

## Summary: Brain Encoding with Proper Alignment

**What we accomplished:**

1. ✅ **Loaded RL model:** Trained PPO agent
2. ✅ **Extracted activations from replays:** Used actual gameplay .bk2 files
3. ✅ **Proper temporal alignment:**
   - Matched replay frames to fMRI TRs using annotations
   - Downsampled from 60Hz to TR (1.49s)
   - Applied HRF convolution
   - Masked non-gameplay periods with NaN
4. ✅ **Applied PCA:** Reduced to 50 components per layer (on valid TRs only)
5. ✅ **Fit encoding models:** Ridge regression with NaN-aware training
6. ✅ **Compared layers:** Identified which layer best predicts brain activity
7. ✅ **Visualized brain maps:** Localized where each layer is encoded

---

### Key Improvements Over Original Approach

**What was fixed:**

1. **Proper replay alignment:**
   - OLD: Agent played arbitrary Level1-1, misaligned with BOLD
   - NEW: Extract activations from exact subject gameplay replays

2. **Temporal alignment:**
   - OLD: No alignment, simple truncation
   - NEW: Use annotation files to map frames → TRs with onset/duration

3. **HRF convolution:**
   - OLD: Missing HRF convolution
   - NEW: Applied SPM HRF after downsampling

4. **Multiple games per run:**
   - OLD: Couldn't handle multiple game segments
   - NEW: Concatenate segments, mask inter-game periods with NaN

5. **NaN handling:**
   - OLD: No way to exclude non-gameplay periods
   - NEW: Valid mask ensures encoding models only train on gameplay TRs

---

### Expected Results

**If brain uses RL-like representations:**

- **Early layers (conv1/2):** Predict visual cortex
  - Edge detection, textures, low-level visual features
  - Occipital lobe activation

- **Middle layers (conv3/4):** Predict parietal/motor cortex
  - Spatial layout, object positions, movement planning
  - Parietal lobe, premotor cortex

- **Late layers (linear):** Predict frontal cortex
  - Value estimates, policy selection, abstract strategy
  - Prefrontal cortex, anterior cingulate

**Hierarchical gradient:**
- R² should increase from early → late layers if brain uses RL features
- Brain maps should show posterior → anterior gradient

---

### Methodological Lessons

**Critical requirements for encoding analysis:**

1. **Proper stimulus alignment:**
   - Use exact stimuli presented to subject
   - Match timing precisely (frame-by-frame if needed)
   - Account for hemodynamic lag (HRF)

2. **Data quality:**
   - Sufficient valid trials (gameplay periods)
   - Good signal-to-noise ratio
   - Proper preprocessing (motion correction, etc.)

3. **Statistical power:**
   - Multiple sessions/subjects for group analysis
   - Cross-validation to avoid overfitting
   - Appropriate regularization (ridge α)

4. **Interpretation:**
   - Compare to baseline models (GLM with behavioral features)
   - Test specific hypotheses about layer-to-region mapping
   - Consider alternative explanations (motion, attention, etc.)

---

### Comparison to GLM (Notebook 01)

**GLM:**
- ✅ Hypothesis-driven (LEFT_THUMB vs RIGHT_THUMB)
- ✅ Simple, interpretable
- ✅ Works with sparse events
- ✅ Found significant effects (contralateral motor control)

**Encoding (This notebook):**
- ✅ Data-driven (learned RL features)
- ✅ Exploratory (discover representations)
- ✅ Tests computational theories
- ⏳ Requires more data and careful alignment

**Both approaches are complementary:**
- GLM: Validate known effects
- Encoding: Discover new representations

---

### Next Steps & Extensions

**To improve these results:**

1. **More data:**
   - Aggregate across multiple sessions
   - Multi-subject analysis
   - Increase gameplay duration

2. **Better features:**
   - Try different layers simultaneously
   - Non-linear encoding (kernel ridge, neural networks)
   - Task-specific features (value, prediction error, etc.)

3. **Control analyses:**
   - Compare to pixel-based features
   - Test against behavioral-only models
   - Permutation testing for significance

4. **Advanced methods:**
   - Hyperalignment across subjects
   - Representational similarity analysis (RSA)
   - Decoding (BOLD → predicted actions)

---

### Research Questions

**This pipeline enables investigating:**

1. **Computational neuroscience:**
   - Do brain and RL agent use similar representations?
   - Where are value and policy encoded in the brain?
   - How do representations change with learning?

2. **Cognitive neuroscience:**
   - How does the brain represent game state?
   - What role does prediction play in decision-making?
   - How are visual and motor systems integrated?

3. **AI alignment:**
   - Can we build agents that think like humans?
   - What makes representations interpretable?
   - How to design human-aligned reward functions?

**This tutorial provides a complete, working pipeline for these investigations!**